In [86]:
# RUN THIS CELL TO PROPERLY HIGHLIGHT THE EXERCISES
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2019-CS109B/master/content/styles/cs109.css").text
HTML(styles)

In [87]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (5,5)

import numpy as np
from scipy.optimize import minimize
from sklearn.utils import shuffle
%matplotlib inline

In [88]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Input
from tensorflow.keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D,\
                                GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC, Precision, Recall, FalsePositives, FalseNegatives, \
                                    TruePositives, TrueNegatives
from tensorflow.keras.regularizers import l2

In [89]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf

tf.keras.backend.clear_session()  # For easy reset of notebook state.

print(tf.__version__)  # You should see a > 2.0.0 here!


2.1.0


In [91]:
from __future__ import absolute_import, division, print_function, unicode_literals
tf.keras.backend.clear_session()  # For easy reset of notebook state.
print(tf.__version__)  # You should see a > 2.0.0 here!
from tf_keras_vis.utils import print_gpus
print_gpus()

2.1.0
0 GPUs


In [93]:
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils import normalize
import tf_keras_vis.utils as utils
from matplotlib import cm
from tf_keras_vis.gradcam import Gradcam

np.random.seed(109)
tf.random.set_seed(109)

In [96]:
#import tensorflow_addons as tfa
#import tensorflow_datasets as tfds

In [37]:
# 1D input
# https://machinelearningmastery.com/convolutional-layers-for-deep-learning-neural-networks/

a = np.asarray([0,0,0,1,1,0,0,0])
display(a.shape, type(a))
a = a.reshape(1,8,1) # since keras ip has 3d for 1D conv layer
a.shape, a.ndim

(8,)

numpy.ndarray

((1, 8, 1), 3)

In [38]:
# single filter with input shape of 3; aka 3 elements wide
# keras refers to filter shape as kernel size

# create model
model = Sequential()
model.add(Conv1D(1,3, input_shape=(8,1)))
# single filter shape 3

In [39]:
# By default filters are initialized with random weights
# can also manually specify weights
# define a filter capable of detecting bumps; bias = 0

weights = [np.asarray([[[0]], [[1]], [[0]]]), np.asarray([0.0])]
# array of shape 3,1,1
model.set_weights(weights)

# weights must be specified in 3D structure - rows, cols, channels
# filter has 1 row, 3 cols and 1 channel

In [40]:
model.get_weights()

[array([[[0.]],
 
        [[1.]],
 
        [[0.]]], dtype=float32),
 array([0.], dtype=float32)]

In [56]:
# apply the single filter to input data
yhat = model.predict(a)
display(yhat)
type(yhat), yhat.shape

array([[[0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.]]], dtype=float32)

(numpy.ndarray, (1, 6, 1))

* Input is 8 element vector
1. 3 element filter is applied to 1st 3 inputs by calculating dot product
2. Filter is then moved along 1 element & repeat
3. Output feature map as 6 elements; input has 8

### Example of a 2D conv layer

* Vertical line detector


In [60]:
# Constrain to 8*8 gray image - Single channel
b = np.asarray([[0, 0, 0, 1, 1, 0, 0, 0],
[0, 0, 0, 1, 1, 0, 0, 0],
[0, 0, 0, 1, 1, 0, 0, 0],
[0, 0, 0, 1, 1, 0, 0, 0],
[0, 0, 0, 1, 1, 0, 0, 0],
[0, 0, 0, 1, 1, 0, 0, 0],
[0, 0, 0, 1, 1, 0, 0, 0],
[0, 0, 0, 1, 1, 0, 0, 0]])

In [64]:
# Input to a Conv 2D layer must be 4D
# Number of Samples, Columns, Rows, Channels
b = b.reshape(1,8,8,1)

In [65]:
# Filter will be a 2D with 3,3 shape

# create model
model = Sequential()
model.add(Conv2D(1,(3,3), input_shape = (8,8,1)))
# single filter with shape 3*3
# input shape - cols, rows, channels

In [73]:
# Define vertical line detector
detector = [
    [
        [[0]],[[1]],[[0]]],
            [[[0]],[[1]],[[0]]],
            [[[0]],[[1]],[[0]]
            ]
]
weights = [np.asarray(detector), np.asarray([0.0])]

# store weights in the model
model.set_weights(weights)

# confirm
print(model.get_weights())

[array([[[[0.]],

        [[1.]],

        [[0.]]],


       [[[0.]],

        [[1.]],

        [[0.]]],


       [[[0.]],

        [[1.]],

        [[0.]]]], dtype=float32), array([0.], dtype=float32)]


In [78]:
# apply the filter to input data
yhat = model.predict(b)
yhat.shape

(1, 6, 6, 1)

In [80]:
# shape of output feature map is 4D
# batch, rows, columns, filters
# we have 1 batch and 1 filter; shape = [1,x,x,1]

# print the content of the single feature map
for r in range(yhat.shape[1]):
    # print each column
    print([yhat[0,r,c,0] for c in range(yhat.shape[2])])

[0.0, 0.0, 3.0, 3.0, 0.0, 0.0]
[0.0, 0.0, 3.0, 3.0, 0.0, 0.0]
[0.0, 0.0, 3.0, 3.0, 0.0, 0.0]
[0.0, 0.0, 3.0, 3.0, 0.0, 0.0]
[0.0, 0.0, 3.0, 3.0, 0.0, 0.0]
[0.0, 0.0, 3.0, 3.0, 0.0, 0.0]


### Padding & Stride

* Subsequent application of filters reduces the size of the image
* 8*8 image --> 3*3 filter applied 2 times gives 4*4 feature map
* This becomes a problem when we develop deep feature maps

In [82]:
# Example of Stacked concolution layers

# create model
model = Sequential()
model.add(Conv2D(1,(3,3), input_shape = (8,8,1)))
model.add(Conv2D(1,(3,3))) # i/p shape needed only for 1st layer
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 6, 6, 1)           10        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 1)           10        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [83]:
# Filter with 1 X 1 size
model = Sequential()
model.add(Conv2D(1,(1,1), input_shape = (8,8,1)))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 8, 8, 1)           2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [84]:
# Filter with 8 X 8 size
model = Sequential()
model.add(Conv2D(1,(8,8), input_shape = (8,8,1)))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 1, 1, 1)           65        
Total params: 65
Trainable params: 65
Non-trainable params: 0
_________________________________________________________________


#### Padding & Strides

* Specified as padding = 'valid' or 'same
* strides = (num, num)

# Lab 6